In [2]:
import cv2
import numpy as np

In [3]:
# tải ảnh
img = cv2.imread('./img/faces.jpg')

# hien thi anh goc
cv2.imshow('Original',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
# tải mô hình đã dc huấn luyện
net = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt','./models/res10_300x300_ssd_iter_140000_fp16.caffemodel')

# chuan bi du lieu dau vao
# img ảnh đầu vào
# 1.x sau mỗi lần quét sẽ scale down xuống x%
# (300,300) kích  thước đầu vào yêu cầu của mô hình
# (102,105,113) màu sắc trung bình
# swapRB=false: ko đổi màu 2 kênh Red và Blue
blob = cv2.dnn.blobFromImage(img, 1.0, (300,300), (102,105,113), swapRB=False)

# dat du lieu dau vao cho dnn
net.setInput(blob)

# chay de phat hien khuon mat
faces = net.forward()

# lay kich thuoc anh ban dau
h = img.shape[0]
w = img.shape[1]
print(h,w)

print(faces.shape)
# voi output (1,1,200,7) từ faces var:
# 1: chiều đầu tiên có kt 1: đại diện cho số batch size của ảnh đầu vào (th này có 1 ảnh nên size là 1)
# 1: Đây là số lớp (classes) hoặc số kích thước (size dimensions) trong dữ liệu đầu ra. Vì đây là mô hình phát hiện khuôn mặt, nó chỉ có một loại đối tượng cần phát hiện—khuôn mặt—nên giá trị này cũng là 1.
# 200: số khuôn mặt tối đa mô hình phát hiện được
# 7: 7 thông số của mỗi khuôn mặt phát hiện dc
    #faces[0,0,i,0]: Giá trị không quan trọng trong mô hình này (thường dùng trong phân loại).
    #faces[0,0,i,1]: Lớp của đối tượng (luôn là 1 vì đây là mô hình nhận diện khuôn mặt).
    #faces[0,0,i,2]: Độ tin cậy (confidence score) - Giá trị từ 0 đến 1 cho biết mô hình có chắc chắn về khuôn mặt này hay không.
    #faces[0,0,i,3]: X_min - Tọa độ x của góc trên bên trái hộp giới hạn (bounding box).
    #faces[0,0,i,4]: Y_min - Tọa độ y của góc trên bên trái.
    #faces[0,0,i,5]: X_max - Tọa độ x của góc dưới bên phải.
    #faces[0,0,i,6]: Y_max - Tọa độ y của góc dưới bên phải.
    
#print(faces[0,0,5,])
# voi output [0. 1. 0.9962274  0.08225593 0.4363947  0.16246499 0.5499648 ]
    # 1. đại diện cho lớp khuôn mặt
    # 0.9962274: độ tin cậy, ~~ 1 => chắc chắn là mặt theo mô hình
    # 4 kq tiếp theo tương ứng 4 tọa độ tương ứng 4 góc phát hiện theo tỉ lệ kích thước thực của từng cạnh là 1

# duyet tung khuon mat trong anh dc phat hien
for i in range(0, faces.shape[2]):
    confi = faces[0,0,i,2] #voi 2 la chi so confidence 
    # kiểm tra i có confidence >= 0.5
    if confi >= 0.5:
        # trich xuat toa do face
        print(faces[0,0,i,3:7])
        # 4 gia tri cuoi cua shape (3:7) cua faces => xác định vị trí khuôn mặt trên ảnh đưa vào
        sx = int(faces[0,0,i,3] * w)  # X_min * width
        sy = int(faces[0,0,i,4] * h)  # Y_min * height
        ex = int(faces[0,0,i,5] * w)  # X_max * width
        ey = int(faces[0,0,i,6] * h)  # Y_max * height
        print(sx,sy,ex,ey)

        # vẽ hình chữ nhật quanh khuôn mặt đã phát hiện
        cv2.rectangle(img, (sx,sy), (ex,ey), (255,255,0), 1)
        # hiển thị độ tin cậy
        text = '{:.2f}%'.format(confi*100)
        cv2.putText(img, text, (sx, sy-10), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255),1)

cv2.imshow('Detect',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

469 660
(1, 1, 200, 7)
[0.085783   0.35571057 0.15895772 0.47616255]
56 166 104 223
[0.30504456 0.32615545 0.36922148 0.45471397]
201 152 243 213
[0.5964006  0.21441548 0.6609324  0.30817837]
393 100 436 144
[0.45469242 0.24470839 0.52321297 0.35918692]
300 114 345 168
